In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
from PIL import Image
import os
import numpy as np
from tqdm.notebook import tqdm
import json

In [ ]:
IMAGE_PATH = "/content/drive/Shareddrives/1:1DivyaKrishna/Data/subset-spark-test/images"
ANNOTATION_PATH = "/content/drive/Shareddrives/1:1DivyaKrishna/Data/subset-spark-test/labels"
NEW_ANNOTATION_PATH = "/content/drive/Shareddrives/1:1DivyaKrishna/Data/subset-spark-test/labels-yolov8"

# Order of bounding boxes
y_min, x_min, y_max, x_max = [121, 258, 783, 1024]

{0: 'cheops', 1: 'debris', 2: 'double_star', 3: 'earth_observation_sat_1', 4: 'lisa_pathfinder', 5: 'proba_2', 6: 'proba_3_csc', 7: 'proba_3_ocs', 8: 'smart_1', 9: 'soho', 10: 'xmm_newton'}


In [ ]:
def create_yolo_coordinates(bbox, img_details, index):
    # extract coordinates
    #x_min, y_min, x_max, y_max = bbox
    y_min, x_min, y_max, x_max = bbox
    width, height = img_details

    # get the x center y center
    bbox_width_normalize = (x_max - x_min)/width
    bbox_height_normalize = (y_max - y_min)/height
    x_center_normalise = ((x_max + x_min)/2)/width
    y_center_normalise = ((y_max + y_min)/2)/height
    #create the coorindate prompt
    cooridinates = f"{index} {x_center_normalise} {y_center_normalise} {bbox_width_normalize} {bbox_height_normalize}"
    return cooridinates

In [ ]:
from tqdm import tqdm
# all the images have same dimension
IMAGE_SIZE = [1024, 1024]
file_list = os.listdir(ANNOTATION_PATH)

for text_file_name in tqdm(file_list):
  input_file_path = ANNOTATION_PATH + '/' + text_file_name
  output_file_path = NEW_ANNOTATION_PATH + '/' + text_file_name
  with open(input_file_path, 'r') as input_file, open(output_file_path, 'w') as output_file:
      # Loop through each line in the input file
      for line in input_file:
          # Split the line to extract the bbox values
          values = line.strip().split()

          # Convert the values to float
          bbox = [float(value) for value in values[1:]]  # Exclude the first value (index)

          # Call the create_yolo_coordinates function to convert the bbox values
          img_details = (IMAGE_SIZE[0], IMAGE_SIZE[1])  # Provide the image width and height
          index = int(values[0])  # Convert the index to an integer
          yolo_coordinates = create_yolo_coordinates(bbox, img_details, index)
          # Write the converted coordinates to the output file
          output_file.write(yolo_coordinates + '\n')

